<a href="https://colab.research.google.com/github/jihyeon602/DeepLearning_practice/blob/main/%EA%B5%B0%EA%B2%BD%EA%B3%84%EC%9E%91%EC%A0%84%ED%99%98%EA%B2%BD%ED%95%A9%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 군 경계 작전 환경 합성 데이터
- 문제정의: **여러 class를 실시간 탐지하는 모델 생성 및 추론 테스트**

- **dataset**: EO(RGB)_SU(하절기)_DT(주간) 데이터셋만 사용  #용량이슈
  - 원본 그대로 TRAIN은 train val로 구성

- **annotations**: 문서에 있는 class만 실시간 탐지하도록 구성
  - GPU 사용 제한 시 샘플링❗

- **산출물**: model파일(.pt 등), results

- 640으로 리사이즈
- whxy 정규화

### 데이터 로드

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile

zip_file_path = '/content/drive/MyDrive/AMY/TL_EO_SU_DT.zip'
extract_to = '/content/drive/MyDrive/AMY/labels_json_all/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [4]:
import zipfile

zip_file_path = '/content/drive/MyDrive/AMY/TS_EO_SU_DT.zip'
extract_to = '/content/drive/MyDrive/AMY/images_all/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [5]:
import zipfile

zip_file_path = '/content/drive/MyDrive/AMY/VL_EO_SU_DT.zip'
extract_to = '/content/drive/MyDrive/AMY/labels_json_all/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [7]:
import zipfile

zip_file_path = '/content/drive/MyDrive/AMY/VS_EO_SU_DT.zip'
extract_to = '/content/drive/MyDrive/AMY/images_all/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


### 데이터 샘플링

- 전체 개수 확인

In [5]:
import os

train_img = '/content/drive/MyDrive/AMY/images_all/train'
train_json = '/content/drive/MyDrive/AMY/labels_json_all/train'
val_img = '/content/drive/MyDrive/AMY/images_all/val'
val_json = '/content/drive/MyDrive/AMY/labels_json_all/val'

image_exts = ['.jpg', '.jpeg', '.png']
json_exts = ['.json']

# 각 폴더에 대해 파일 개수 계산
image_train = [f for f in os.listdir(train_img) if os.path.splitext(f)[-1].lower() in image_exts]
json_train = [f for f in os.listdir(train_json) if os.path.splitext(f)[-1].lower() in json_exts]
image_val = [f for f in os.listdir(val_img) if os.path.splitext(f)[-1].lower() in image_exts]
json_val = [f for f in os.listdir(val_json) if os.path.splitext(f)[-1].lower() in json_exts]

# 결과 출력
print(f"train 이미지 파일 개수: {len(image_train)}개")
print(f"train JSON 파일 개수: {len(json_train)}개")
print(f"val 이미지 파일 개수: {len(image_val)}개")
print(f"val JSON 파일 개수: {len(json_val)}개")

train 이미지 파일 개수: 28530개
train JSON 파일 개수: 28530개
val 이미지 파일 개수: 3567개
val JSON 파일 개수: 3567개


### 샘플링
- train: 클래스당 100 개
- val: 클래스 당 20개

In [8]:
import os
import json
import shutil
import random

def random_sample(src_img_dir, src_json_dir, dst_img_dir, dst_json_dir, total_sample_count):
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_json_dir, exist_ok=True)

    # 전체 JSON 파일 수집
    all_files = [f for f in os.listdir(src_json_dir) if f.endswith('.json')]

    print(f"전체 파일 개수: {len(all_files)}개")

    # 무작위 섞기
    random.shuffle(all_files)

    # 샘플링
    sampled = all_files[:total_sample_count]

    for json_file in sampled:
        base = os.path.splitext(json_file)[0]
        src_img = os.path.join(src_img_dir, base + '.jpg')
        src_json = os.path.join(src_json_dir, json_file)

        dst_img = os.path.join(dst_img_dir, base + '.jpg')
        dst_json = os.path.join(dst_json_dir, json_file)

        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)
            shutil.copy(src_json, dst_json)

    print(f" 랜덤 샘플링 완료 ({len(sampled)}개)")

# 🔹 train 랜덤 샘플링
random_sample(
    src_img_dir='/content/drive/MyDrive/AMY/images_all/train',
    src_json_dir='/content/drive/MyDrive/AMY/labels_json_all/train',
    dst_img_dir='/content/drive/MyDrive/AMY/images/train',
    dst_json_dir='/content/drive/MyDrive/AMY/labels_json/train',
    total_sample_count=300  # 전체 중 300개 랜덤 샘플링
)

# 🔹 val 랜덤 샘플링
random_sample(
    src_img_dir='/content/drive/MyDrive/AMY/images_all/val',
    src_json_dir='/content/drive/MyDrive/AMY/labels_json_all/val',
    dst_img_dir='/content/drive/MyDrive/AMY/images/val',
    dst_json_dir='/content/drive/MyDrive/AMY/labels_json/val',
    total_sample_count=50  # 전체 중 50개 랜덤 샘플링
)


전체 파일 개수: 28530개
 랜덤 샘플링 완료 (300개)
전체 파일 개수: 3567개
 랜덤 샘플링 완료 (50개)


### json 파일 yolo 포맷(.txt) 파일로 변환

- w
- h
- x
- y

In [ ]:
import json
from PIL import Image

# 기본 경로
base_path = "/content/drive/MyDrive/AMY"

# 경로 딕셔너리
sets = {
    'train': {
        'json_folder': os.path.join(base_path, "labels_json", "train"),   # JSON 라벨
        'image_folder': os.path.join(base_path, "images", "train"),  # 이미지
        'output_folder': os.path.join(base_path, "labels", "train")  # YOLO 형식 저장될 위치
    },
    'val': {
        'json_folder': os.path.join(base_path, "labels_json", "val"),
        'image_folder': os.path.join(base_path, "images", "val"),
        'output_folder': os.path.join(base_path, "labels", "val")
    }
}

subclass_to_idx = {
    11: 0, 12: 1, 13: 2,
    21: 3, 22: 4, 23: 5,
    31: 6, 41: 7, 42: 8
}

def convert_to_yolo(json_folder, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(json_folder):
        if file.endswith('.json'):
            base_name = os.path.splitext(file)[0]

            with open(os.path.join(json_folder, file), 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 이미지 경로 찾기
            image_path = None
            for ext in ['.jpg', '.jpeg', '.png']:
                candidate = os.path.join(image_folder, base_name + ext)
                if os.path.exists(candidate):
                    image_path = candidate
                    break

            if not image_path:
                print(f"⚠️ 이미지 없음: {base_name}")
                continue

            # 이미지 열기
            try:
                with Image.open(image_path) as img:
                    img_width, img_height = img.size
            except Exception as e:
                print(f"⚠️ 이미지 열기 실패: {image_path} - {e}")
                continue

            # YOLO 라벨 변환
            yolo_lines = []
            for ann in data.get("annotations", []):
                bbox = ann.get("bounding_box")
                if bbox and len(bbox) == 4:
                    # assuming bbox format is [w, h, x, y]
                    w, h, x, y = bbox

                    # 중앙 좌표 및 크기 계산 (이미지 크기 그대로 사용)
                    cx = (x + w / 2) / img_width  # 중심 좌표 X
                    cy = (y + h / 2) / img_height  # 중심 좌표 Y
                    w = w / img_width  # 너비 비율
                    h = h / img_height  # 높이 비율

                    # sub_class 값으로 클래스 id 결정
                    sub_class = ann.get("sub_class", 0)
                    class_id = subclass_to_idx.get(sub_class, -1)
                    if class_id == -1:
                        print(f"⚠️ 잘못된 sub_class 값: {sub_class} - {base_name}")
                        continue

                    # YOLO 형식으로 라인 추가
                    yolo_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

            # 결과 저장
            out_path = os.path.join(output_folder, base_name + ".txt")
            with open(out_path, "w", encoding='utf-8') as f:
                f.write("\n".join(yolo_lines))

            print(f"변환 완료: {base_name}.txt")

# 실행
for phase, paths in sets.items():
    print(f"\n📂 처리 중: {phase}")
    convert_to_yolo(paths['json_folder'], paths['image_folder'], paths['output_folder'])


### 리사이징

In [10]:
from PIL import Image

def resize_images_in_directory(img_dir, target_size=(640, 640)):
    for image_file in os.listdir(img_dir):
        if image_file.lower().endswith(('.jpg')):  # 이미지 파일만 처리
            image_path = os.path.join(img_dir, image_file)

            with Image.open(image_path) as img:
                # 리사이즈
                img_resized = img.resize(target_size)
                # 덮어쓰기 저장
                img_resized.save(image_path)

# 경로 지정
train_img_dir = '/content/drive/MyDrive/AMY/images/train'
val_img_dir = '/content/drive/MyDrive/AMY/images/val'

# 실행
resize_images_in_directory(train_img_dir)
resize_images_in_directory(val_img_dir)

print("리사이징 완료")

리사이징 완료


### yaml 파일 제작

어선=11, 군함=12, 상선=13, 고정익 유인항공기=21, 회전익 유인항공기=22 무인항공기=23, 새=31, 삐라=41, 오물폭탄=42

In [16]:
# YAML 파일 내용
yaml_content = """
train: /content/drive/MyDrive/AMY/images/train
val: /content/drive/MyDrive/AMY/images/val

nc: 9
names: ["11_어선", "12_군함", "13_상선", "21_고정유인항공기", "22_회전유인항공기", "23_무인항공기", "31_새", "41_삐라", "42_오물폭탄"]
"""

# 파일 저장
with open('/content/drive/MyDrive/AMY/classify.yaml', 'w') as file:
    file.write(yaml_content)

print("classify.yaml 파일 생성완료.")

classify.yaml 파일 생성완료.


## yolo 모델 구현
- yolo11m.pt

In [12]:
!git clone https://github.com/ultralytics/yolo11

Cloning into 'yolo11'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!pip install ultralytics

In [ ]:
!yolo detect train data='/content/drive/MyDrive/AMY/classify.yaml' model=yolo11n.pt epochs=50 imgsz=640 patience=10 batch=8 seed=42

Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/MyDrive/AMY/classify.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 